In [1]:
# import dependencies
import pandas as pd
import requests
import numpy as np

In [2]:
# links
url1 = 'https://www.boxofficemojo.com/year/2023/?ref_=bo_yl_table_1'
url2 = 'https://www.boxofficemojo.com/holiday/?ref_=bo_nb_yld_secondarytab'
url3 = 'https://www.boxofficemojo.com/chart/mpaa_title_lifetime_gross/?by_mpaa=PG-13&ref_=bo_cso_ac'
# Holidays 2019-2022
url_2019 = 'https://www.boxofficemojo.com/holiday/by-year/2019/?grossesOption=calendarGrosses'
url_2020 = 'https://www.boxofficemojo.com/holiday/by-year/2020/?grossesOption=calendarGrosses'
url_2021 = 'https://www.boxofficemojo.com/holiday/by-year/2021/?grossesOption=calendarGrosses'
url_2022 = 'https://www.boxofficemojo.com/holiday/by-year/2022/?grossesOption=calendarGrosses'


# retrieve html for links
response1 = requests.get(url1)
response2 = requests.get(url2)
response3 = requests.get(url3)
response4 = requests.get(url_2019)
response5 = requests.get(url_2020)
response6 = requests.get(url_2021)
response7 = requests.get(url_2022)

# Read responses into tables
top_domestic_2023 = pd.read_html(response1.text)
holiday_revenue_2023 = pd.read_html(response2.text)
top_200_alltime = pd.read_html(response3.text)
holidays_2019 = pd.read_html(response4.text)
holidays_2020 = pd.read_html(response5.text)
holidays_2021 = pd.read_html(response6.text)
holidays_2022 = pd.read_html(response7.text)

In [3]:
# Select and display first 3 tables
top_dom_2023_df = top_domestic_2023[0]
holiday_rev_2023_df = holiday_revenue_2023[0]
top_200_alltime_df = top_200_alltime[0]

display(top_dom_2023_df.head(3))
display(holiday_rev_2023_df.head(3))
display(top_200_alltime_df.head(3))

,Rank,Release,Genre,Budget,Running Time,Gross,Theaters,Total Gross,Release Date,Distributor,Estimated
0,1,Barbie,-,-,-,"$636,225,983",4337,"$636,227,208",Jul 21,Warner Bros.,False
1,2,The Super Mario Bros. Movie,-,-,-,"$574,934,330",4371,"$574,934,330",Apr 5,Universal Pictures,False
2,3,Spider-Man: Across the Spider-Verse,-,-,-,"$381,311,319",4332,"$381,311,319",Jun 2,Columbia Pictures,False


,Holiday,Cumulative Gross,% of Year,Releases,Average,#1 Release,Genre,Budget,Running Time,Gross,% of Total
0,New Year's Eve,"$26,251,235",0.3%,36,"$729,200",Wonka,-,-,-,"$5,208,897",19.8%
1,Christmas Day,"$63,107,349",0.7%,39,"$1,618,137",The Color Purple,-,-,-,"$18,151,050",28.8%
2,Christmas Weekend,"$157,152,539",1.8%,41,"$3,832,988",Aquaman and the Lost Kingdom,-,-,-,"$38,133,816",24.3%


,Title,Rank,Lifetime Gross,Overall Rank,Year
0,Star Wars: Episode VII - The Force Awakens,1,"$936,662,225",1,2015
1,Avengers: Endgame,2,"$858,373,000",2,2019
2,Spider-Man: No Way Home,3,"$814,115,070",3,2021


### Table - Top Domestic Movies of 2023 
---

In [4]:
top_dom_2023_df.columns

Index(['Rank', 'Release', 'Genre', 'Budget', 'Running Time', 'Gross',
       'Theaters', 'Total Gross', 'Release Date', 'Distributor', 'Estimated'],
      dtype='object')

In [5]:
top_dom_2023_df['Estimated'].value_counts()

Estimated
False    200
Name: count, dtype: int64

In [6]:
# remove Genre, Budget, Running Time, and Estimated
top_dom_2023_df = top_dom_2023_df[['Rank', 'Release', 'Gross','Theaters', 'Total Gross', 'Distributor']]

In [7]:
top_dom_2023_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Rank         200 non-null    int64 
 1   Release      200 non-null    object
 2   Gross        200 non-null    object
 3   Theaters     200 non-null    object
 4   Total Gross  200 non-null    object
 5   Distributor  200 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.5+ KB


In [8]:
top_200 = top_dom_2023_df.copy()

In [9]:
top_200['Total Gross'].apply(type).value_counts()

Total Gross
<class 'str'>    200
Name: count, dtype: int64

In [10]:
# Convert 'Gross' and 'Total Gross'

# top_200['Gross'].apply(type).value_counts()

top_200['Gross'] = top_200['Gross'].str.replace(',', '')
top_200['Gross'] = top_200['Gross'].str.replace('$', '')
top_200['Gross'] = top_200['Gross'].astype('int64', copy=True)

top_200['Total Gross'] = top_200['Total Gross'].str.replace(',', '')
top_200['Total Gross'] = top_200['Total Gross'].str.replace('$', '')
top_200['Total Gross'] = top_200['Total Gross'].astype('int64', copy=True)

top_200[['Gross', 'Total Gross']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Gross        200 non-null    int64
 1   Total Gross  200 non-null    int64
dtypes: int64(2)
memory usage: 3.2 KB


In [11]:
top_200['Theaters'].apply(type).value_counts()

Theaters
<class 'str'>    200
Name: count, dtype: int64

In [12]:
# Convert 'Theaters' column to float, nan values cannot be converted to integer.
theater_list = top_200['Theaters'].values.tolist()

new_list = []
for i in range(len(theater_list)):
    if theater_list[i] == '-':
        new_list.append(np.nan)
    else:
        new_list.append(int(theater_list[i]))

top_200['Theaters'] = new_list

top_200.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Rank         200 non-null    int64  
 1   Release      200 non-null    object 
 2   Gross        200 non-null    int64  
 3   Theaters     198 non-null    float64
 4   Total Gross  200 non-null    int64  
 5   Distributor  200 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.5+ KB


In [13]:
# Rename the Columns 

In [14]:
# Export as csv
# top_200.to_csv('top_dom_200_2023.csv')

### Holidays Table
---

In [15]:
holiday_rev_2023_df.columns

Index(['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Genre', 'Budget', 'Running Time', 'Gross', '% of Total'],
      dtype='object')

In [16]:
holiday_rev_2023_df.head(2)

# Convert Average, % of Year, cumulative gross, gross, % of Total

,Holiday,Cumulative Gross,% of Year,Releases,Average,#1 Release,Genre,Budget,Running Time,Gross,% of Total
0,New Year's Eve,"$26,251,235",0.3%,36,"$729,200",Wonka,-,-,-,"$5,208,897",19.8%
1,Christmas Day,"$63,107,349",0.7%,39,"$1,618,137",The Color Purple,-,-,-,"$18,151,050",28.8%


In [17]:
holiday_df = holiday_rev_2023_df.copy()

In [18]:
# Convert 'Cumulative Gross', 'Average', 'Gross' from str to int

holiday_df['Cumulative Gross'] = holiday_df['Cumulative Gross'].str.replace(',', '')
holiday_df['Cumulative Gross'] = holiday_df['Cumulative Gross'].str.replace('$', '')
holiday_df['Cumulative Gross'] = holiday_df['Cumulative Gross'].astype('int64', copy=True)

holiday_df['Average'] = holiday_df['Average'].str.replace(',', '')
holiday_df['Average'] = holiday_df['Average'].str.replace('$', '')
holiday_df['Average'] = holiday_df['Average'].astype('int64', copy=True)

holiday_df['Gross'] = holiday_df['Gross'].str.replace(',', '')
holiday_df['Gross'] = holiday_df['Gross'].str.replace('$', '')
holiday_df['Gross'] = holiday_df['Gross'].astype('int64', copy=True)


holiday_df[['Cumulative Gross', 'Average', 'Gross']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Cumulative Gross  25 non-null     int64
 1   Average           25 non-null     int64
 2   Gross             25 non-null     int64
dtypes: int64(3)
memory usage: 728.0 bytes


In [19]:
holiday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Holiday           25 non-null     object
 1   Cumulative Gross  25 non-null     int64 
 2   % of Year         25 non-null     object
 3   Releases          25 non-null     int64 
 4   Average           25 non-null     int64 
 5   #1 Release        25 non-null     object
 6   Genre             25 non-null     object
 7   Budget            25 non-null     object
 8   Running Time      25 non-null     object
 9   Gross             25 non-null     int64 
 10  % of Total        25 non-null     object
dtypes: int64(4), object(7)
memory usage: 2.3+ KB


In [20]:
# Convert columns '% of Year' and '% of Total' to float

holiday_df['% of Year'] = holiday_df['% of Year'].str.replace('%', '')
holiday_df['% of Year'] = holiday_df['% of Year'].astype(float)/100

holiday_df['% of Total'] = holiday_df['% of Total'].str.replace('%', '')
holiday_df['% of Total'] = holiday_df['% of Total'].astype(float)/100


holiday_df[['% of Year', '% of Total']].head()

,% of Year,% of Total
0,0.003,0.198
1,0.007,0.288
2,0.018,0.243
3,0.011,0.224
4,0.013,0.250


In [21]:
# Remove columns without data

display(holiday_df.columns)

holiday_df = holiday_df[['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Gross', '% of Total']]

display(holiday_df.columns)

Index(['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Genre', 'Budget', 'Running Time', 'Gross', '% of Total'],
      dtype='object')

Index(['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Gross', '% of Total'],
      dtype='object')

In [22]:
holiday_list = holiday_df['Holiday'].values.tolist()

holiday_list

["New Year's Eve",
 'Christmas Day',
 'Christmas Weekend',
 'Post-Thanksgiving Weekend',
 'Thanksgiving Weekend',
 'Thanksgiving 4-Day Weekend',
 'Thanksgiving',
 'Thanksgiving 5-Day Weekend',
 'Halloween',
 "Indigenous People's Day",
 "Indigenous People's Day Weekend",
 'Labor Day',
 'Labor Day Weekend',
 'Independence Day',
 'Independence Day Weekend',
 'Memorial Day',
 'Memorial Day Weekend',
 'Easter',
 'Easter Weekend',
 "President's Day",
 "President's Day Weekend",
 'MLK Day',
 'MLK Day Weekend',
 "New Year's Day",
 'New Year Weekend']

In [23]:
# Encode each holiday as a month
# holiday month list
month_list = []
month_name_list = []

# loop thru list
for i in range(len(holiday_list)):
    if i <= 4:
        month_list.append(11)
        month_name_list.append('November')
    elif i <= 7:
        month_list.append(10)
        month_name_list.append('October')
    elif i <= 9:
        month_list.append(9)
        month_name_list.append('September')
    elif i <= 11:
        month_list.append(7)
        month_name_list.append('July')
    elif i  <= 13:
        month_list.append(5)
        month_name_list.append('May')
    elif i <= 15: 
        month_list.append(4)
        month_name_list.append('April')
    elif i <= 17: 
        month_list.append(2)
        month_name_list.append('February')
    else:
        month_list.append(1)
        month_name_list.append('January')
        
display(len(month_list))
display(len(month_name_list))

# Add lists to df as columns
holiday_df['num_month'] = month_list
holiday_df['month'] = month_name_list

holiday_df.head()

25

25

,Holiday,Cumulative Gross,% of Year,Releases,Average,#1 Release,Gross,% of Total,num_month,month
0,New Year's Eve,26251235,0.003,36,729200,Wonka,5208897,0.198,11,November
1,Christmas Day,63107349,0.007,39,1618137,The Color Purple,18151050,0.288,11,November
2,Christmas Weekend,157152539,0.018,41,3832988,Aquaman and the Lost Kingdom,38133816,0.243,11,November
3,Post-Thanksgiving Weekend,97110956,0.011,51,1904136,Renaissance: A Film by Beyoncé,21801216,0.224,11,November
4,Thanksgiving Weekend,115964224,0.013,53,2188004,The Hunger Games: The Ballad of Songbirds & Sn...,29042517,0.250,11,November


In [24]:
holiday_df.columns

Index(['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Gross', '% of Total', 'num_month', 'month'],
      dtype='object')

In [25]:
# Rearrange order of columns
display(holiday_df.columns)

holiday_df = holiday_df[['Holiday', 'month', 'num_month', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Gross', '% of Total']]

display(holiday_df.columns)

Index(['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Gross', '% of Total', 'num_month', 'month'],
      dtype='object')

Index(['Holiday', 'month', 'num_month', 'Cumulative Gross', '% of Year',
       'Releases', 'Average', '#1 Release', 'Gross', '% of Total'],
      dtype='object')

In [26]:
# Rename the Columns 
# holiday_df.columns = ['holiday', 'month', 'num_month', 'cumulative_gross', 'percent_of_year', 'releases', 'average', 'top_release', 'gross', 'percent_of_total']

# holiday_df.columns

In [27]:
# Export to csv
# holiday_df.to_csv('holiday_revenue_2023.csv')

### Top 200 Movies of Alltime

In [28]:
top_200_alltime_df.head(3)

,Title,Rank,Lifetime Gross,Overall Rank,Year
0,Star Wars: Episode VII - The Force Awakens,1,"$936,662,225",1,2015
1,Avengers: Endgame,2,"$858,373,000",2,2019
2,Spider-Man: No Way Home,3,"$814,115,070",3,2021


In [29]:
top_200_alltime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Title           200 non-null    object
 1   Rank            200 non-null    int64 
 2   Lifetime Gross  200 non-null    object
 3   Overall Rank    200 non-null    int64 
 4   Year            200 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.9+ KB


In [30]:
alltime_df = top_200_alltime_df.copy()

In [31]:
# Convert 'Lifetime Gross' column to integer type

alltime_df['Lifetime Gross'] = alltime_df['Lifetime Gross'].str.replace(',', '')
alltime_df['Lifetime Gross'] = alltime_df['Lifetime Gross'].str.replace('$', '')
alltime_df['Lifetime Gross'] = alltime_df['Lifetime Gross'].astype('int64', copy=True)

alltime_df.head(3)

,Title,Rank,Lifetime Gross,Overall Rank,Year
0,Star Wars: Episode VII - The Force Awakens,1,936662225,1,2015
1,Avengers: Endgame,2,858373000,2,2019
2,Spider-Man: No Way Home,3,814115070,3,2021


In [32]:
# Export to csv
# alltime_df.to_csv('top_200_alltime.csv')

In [33]:
alltime_df.describe()

,Rank,Lifetime Gross,Overall Rank,Year
count,200.000000,2.000000e+02,200.000000,200.000000
mean,100.500000,2.855036e+08,172.250000,2009.875000
std,57.879185,1.446265e+08,106.109536,8.401925
min,1.000000,1.624344e+08,1.000000,1986.000000
25%,50.750000,1.842011e+08,79.750000,2004.000000
50%,100.500000,2.289015e+08,171.500000,2011.000000
75%,150.250000,3.312711e+08,267.250000,2016.250000
max,200.000000,9.366622e+08,358.000000,2023.000000


### Other Holiday Tables
---

In [34]:
holidays_2019_df = holidays_2019[0]
holidays_2020_df = holidays_2020[0]
holidays_2021_df = holidays_2021[0]
holidays_2022_df = holidays_2022[0]

display(len(holidays_2019_df), len(holidays_2020_df), len(holidays_2021_df), len(holidays_2022_df))

22

22

23

25

In [35]:
# Holidays are different year to year(?)
display(holidays_2019_df['Holiday'].values)
display(holidays_2020_df['Holiday'].values)
display(holidays_2021_df['Holiday'].values)
display(holidays_2022_df['Holiday'].values)

array(["New Year's Eve", 'Christmas Day', 'Post-Thanksgiving Weekend',
       'Thanksgiving Weekend', 'Thanksgiving 4-Day Weekend',
       'Thanksgiving', 'Thanksgiving 5-Day Weekend', 'Halloween',
       "Indigenous People's Day", "Indigenous People's Day Weekend",
       'Labor Day', 'Labor Day Weekend', 'Independence Day',
       'Memorial Day', 'Memorial Day Weekend', 'Easter', 'Easter Weekend',
       "President's Day", "President's Day Weekend", 'MLK Day',
       'MLK Day Weekend', "New Year's Day"], dtype=object)

array(["New Year's Eve", 'Christmas Day', 'Post-Thanksgiving Weekend',
       'Thanksgiving Weekend', 'Thanksgiving 4-Day Weekend',
       'Thanksgiving', 'Thanksgiving 5-Day Weekend', 'Halloween',
       "Indigenous People's Day", "Indigenous People's Day Weekend",
       'Labor Day', 'Labor Day Weekend', 'Independence Day',
       'Memorial Day', 'Memorial Day Weekend', 'Easter', 'Easter Weekend',
       "President's Day", "President's Day Weekend", 'MLK Day',
       'MLK Day Weekend', "New Year's Day"], dtype=object)

array(["New Year's Eve", 'Christmas Day', 'Post-Thanksgiving Weekend',
       'Thanksgiving Weekend', 'Thanksgiving 4-Day Weekend',
       'Thanksgiving', 'Thanksgiving 5-Day Weekend', 'Halloween',
       "Indigenous People's Day", "Indigenous People's Day Weekend",
       'Labor Day', 'Labor Day Weekend', 'Independence Day',
       'Independence Day Weekend', 'Memorial Day', 'Memorial Day Weekend',
       'Easter', 'Easter Weekend', "President's Day",
       "President's Day Weekend", 'MLK Day', 'MLK Day Weekend',
       "New Year's Day"], dtype=object)

array(["New Year's Eve", 'Christmas Day', 'Christmas Weekend',
       'Post-Thanksgiving Weekend', 'Thanksgiving Weekend',
       'Thanksgiving 4-Day Weekend', 'Thanksgiving',
       'Thanksgiving 5-Day Weekend', 'Halloween',
       "Indigenous People's Day", "Indigenous People's Day Weekend",
       'Labor Day', 'Labor Day Weekend', 'Independence Day',
       'Independence Day Weekend', 'Memorial Day', 'Memorial Day Weekend',
       'Easter', 'Easter Weekend', "President's Day",
       "President's Day Weekend", 'MLK Day', 'MLK Day Weekend',
       "New Year's Day", 'New Year Weekend'], dtype=object)

In [36]:
holidays_2019_df.head(1)

,Holiday,Cumulative Gross,% of Year,Releases,Average,#1 Release,Genre,Budget,Running Time,Gross,% of Total
0,New Year's Eve,"$43,962,854",0.4%,54,"$814,126",Star Wars: Episode IX - The Rise of Skywalker,-,-,-,"$13,217,698",30.1%


In [37]:
holidays_2019_df.columns

Index(['Holiday', 'Cumulative Gross', '% of Year', 'Releases', 'Average',
       '#1 Release', 'Genre', 'Budget', 'Running Time', 'Gross', '% of Total'],
      dtype='object')

In [38]:
# Trim the columns to Holiday, Cumulative Gross, % of Year, Releases, Average
holidays_2019_df = holidays_2019_df[['Holiday', 'Cumulative Gross', 'Releases', 'Average']]
holidays_2020_df = holidays_2020_df[['Holiday', 'Cumulative Gross', 'Releases', 'Average']]
holidays_2021_df = holidays_2021_df[['Holiday', 'Cumulative Gross', 'Releases', 'Average']]
holidays_2022_df = holidays_2022_df[['Holiday', 'Cumulative Gross', 'Releases', 'Average']]

In [39]:
df_2019 = holidays_2019_df.copy()

In [40]:
# Convert each Table
df_2019['Cumulative Gross'] = df_2019['Cumulative Gross'].str.replace(',', '')
df_2019['Cumulative Gross'] = df_2019['Cumulative Gross'].str.replace('$', '')
df_2019['Cumulative Gross'] = df_2019['Cumulative Gross'].astype('int64', copy=True)
df_2019['Average'] = df_2019['Average'].str.replace(',', '')
df_2019['Average'] = df_2019['Average'].str.replace('$', '')
df_2019['Average'] = df_2019['Average'].astype('int64', copy=True)

df_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Holiday           22 non-null     object
 1   Cumulative Gross  22 non-null     int64 
 2   Releases          22 non-null     int64 
 3   Average           22 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 832.0+ bytes


In [41]:
df_2019

,Holiday,Cumulative Gross,Releases,Average
0,New Year's Eve,43962854,54,814126
1,Christmas Day,78632563,54,1456158
2,Post-Thanksgiving Weekend,90326095,87,1038230
3,Thanksgiving Weekend,180855568,90,2009506
4,Thanksgiving 4-Day Weekend,215903776,54,3998218
5,Thanksgiving,35370976,52,680211
6,Thanksgiving 5-Day Weekend,262269314,58,4521884
7,Halloween,9536976,54,176610
8,Indigenous People's Day,20973935,56,374534
9,Indigenous People's Day Weekend,161052554,59,2729704


In [42]:
# Drop redundant rows
df_2019_trim = df_2019.drop([2,3,4,5,8,10,13,15,17,19])

df_2019_trim

,Holiday,Cumulative Gross,Releases,Average
0,New Year's Eve,43962854,54,814126
1,Christmas Day,78632563,54,1456158
6,Thanksgiving 5-Day Weekend,262269314,58,4521884
7,Halloween,9536976,54,176610
9,Indigenous People's Day Weekend,161052554,59,2729704
11,Labor Day Weekend,120764144,82,1472733
12,Independence Day,46565065,54,862316
14,Memorial Day Weekend,230819376,97,2379581
16,Easter Weekend,121560823,74,1642713
18,President's Day Weekend,152447001,86,1772639


In [43]:
df_2019_trim.reset_index(inplace=True)

df_2019_trim

,index,Holiday,Cumulative Gross,Releases,Average
0,0,New Year's Eve,43962854,54,814126
1,1,Christmas Day,78632563,54,1456158
2,6,Thanksgiving 5-Day Weekend,262269314,58,4521884
3,7,Halloween,9536976,54,176610
4,9,Indigenous People's Day Weekend,161052554,59,2729704
5,11,Labor Day Weekend,120764144,82,1472733
6,12,Independence Day,46565065,54,862316
7,14,Memorial Day Weekend,230819376,97,2379581
8,16,Easter Weekend,121560823,74,1642713
9,18,President's Day Weekend,152447001,86,1772639


In [44]:
df_2019_trim.columns

Index(['index', 'Holiday', 'Cumulative Gross', 'Releases', 'Average'], dtype='object')

In [45]:
df_2019_trim = df_2019_trim[['Holiday', 'Cumulative Gross', 'Releases', 'Average']]

df_2019_trim

,Holiday,Cumulative Gross,Releases,Average
0,New Year's Eve,43962854,54,814126
1,Christmas Day,78632563,54,1456158
2,Thanksgiving 5-Day Weekend,262269314,58,4521884
3,Halloween,9536976,54,176610
4,Indigenous People's Day Weekend,161052554,59,2729704
5,Labor Day Weekend,120764144,82,1472733
6,Independence Day,46565065,54,862316
7,Memorial Day Weekend,230819376,97,2379581
8,Easter Weekend,121560823,74,1642713
9,President's Day Weekend,152447001,86,1772639


In [46]:
df_2019_trim['Holiday'].iloc[1]

'Christmas Day'

In [47]:
# Encode each holiday as a month
# holiday month list
month_list = []
month_name_list = []

# loop thru list
for i in range(len(df_2019_trim)):
    if df_2019_trim['Holiday'].iloc[i] in ["New Year's Eve", "Christmas Day"]:
        month_list.append(12)
        month_name_list.append('December')
    elif df_2019_trim['Holiday'].iloc[i] in ["Thanksgiving 5-Day Weekend"]:
        month_list.append(11)
        month_name_list.append('November')
    elif df_2019_trim['Holiday'].iloc[i] in ["Halloween", "Indigenous People's Day Weekend"]:
        month_list.append(10)
        month_name_list.append('October')
    elif df_2019_trim['Holiday'].iloc[i] in ["Labor Day Weekend"]:
        month_list.append(9)
        month_name_list.append('September')
    elif df_2019_trim['Holiday'].iloc[i] in ['Independence Day']:
        month_list.append(7)
        month_name_list.append('July')
    elif df_2019_trim['Holiday'].iloc[i] in ['Memorial Day Weekend']: 
        month_list.append(5)
        month_name_list.append('May')
    elif df_2019_trim['Holiday'].iloc[i] in ['Easter Weekend']: 
        month_list.append(4)
        month_name_list.append('April')
    elif df_2019_trim['Holiday'].iloc[i] in ["President's Day Weekend"]: 
        month_list.append(2)
        month_name_list.append('February')
    else:
        month_list.append(1)
        month_name_list.append('January')
        
display(len(month_list))
display(len(month_name_list))

# Add lists to df as columns
df_2019_trim['Month'] = month_name_list
df_2019_trim['Year'] = [int(2019) for x in range(len(df_2019_trim))]

df_2019_trim = df_2019_trim[['Month', 'Year', 'Holiday', 'Cumulative Gross', 'Releases', 'Average']]

df_2019_trim

12

12

,Month,Year,Holiday,Cumulative Gross,Releases,Average
0,December,2019,New Year's Eve,43962854,54,814126
1,December,2019,Christmas Day,78632563,54,1456158
2,November,2019,Thanksgiving 5-Day Weekend,262269314,58,4521884
3,October,2019,Halloween,9536976,54,176610
4,October,2019,Indigenous People's Day Weekend,161052554,59,2729704
5,September,2019,Labor Day Weekend,120764144,82,1472733
6,July,2019,Independence Day,46565065,54,862316
7,May,2019,Memorial Day Weekend,230819376,97,2379581
8,April,2019,Easter Weekend,121560823,74,1642713
9,February,2019,President's Day Weekend,152447001,86,1772639


In [48]:
df_month_2019 = df_2019_trim[['Month', 'Cumulative Gross', 'Releases']].groupby(by='Month').sum()

df_month_2019['Year'] = [int(2019) for x in range(len(df_month_2019))]

df_month_2019 = df_month_2019.reset_index()

df_month_2019 = df_month_2019[['Month', 'Year', 'Cumulative Gross', 'Releases']]

df_month_2019 = df_month_2019.rename(columns = {'Cumulative Gross': 'Holiday Gross', 'Releases': 'Holiday Releases'})

df_month_2019

,Month,Year,Holiday Gross,Holiday Releases
0,April,2019,121560823,74
1,December,2019,122595417,108
2,February,2019,152447001,86
3,January,2019,221945359,122
4,July,2019,46565065,54
5,May,2019,230819376,97
6,November,2019,262269314,58
7,October,2019,170589530,113
8,September,2019,120764144,82


### 2020 Holiday Table

In [49]:
holidays_2020_df

,Holiday,Cumulative Gross,Releases,Average
0,New Year's Eve,"$2,933,369",17,"$172,551"
1,Christmas Day,"$10,214,882",27,"$378,328"
2,Post-Thanksgiving Weekend,"$8,397,393",41,"$204,814"
3,Thanksgiving Weekend,"$14,152,936",41,"$345,193"
4,Thanksgiving 4-Day Weekend,"$17,660,248",29,"$608,974"
5,Thanksgiving,"$3,570,668",27,"$132,246"
6,Thanksgiving 5-Day Weekend,"$20,251,599",36,"$562,544"
7,Halloween,"$3,791,211",35,"$108,320"
8,Indigenous People's Day,"$1,001,260",29,"$34,526"
9,Indigenous People's Day Weekend,"$11,259,812",32,"$351,869"


In [50]:
# Drop redundant holiday rows
df_2020_trim = holidays_2020_df.drop([2,3,4,5,8,10,13,15,17,19])
# Reset Index
df_2020_trim.reset_index(inplace=True)
# Trim the columns
df_2020_trim = df_2020_trim[['Holiday', 'Cumulative Gross', 'Releases']]

df_2020_trim

,Holiday,Cumulative Gross,Releases
0,New Year's Eve,"$2,933,369",17
1,Christmas Day,"$10,214,882",27
2,Thanksgiving 5-Day Weekend,"$20,251,599",36
3,Halloween,"$3,791,211",35
4,Indigenous People's Day Weekend,"$11,259,812",32
5,Labor Day Weekend,"$19,858,242",25
6,Independence Day,"$74,261",9
7,Memorial Day Weekend,"$240,390",6
8,Easter Weekend,"$3,675",2
9,President's Day Weekend,"$182,920,720",78


In [51]:
# Convert Table
df_2020_trim['Cumulative Gross'] = df_2020_trim['Cumulative Gross'].str.replace(',', '')
df_2020_trim['Cumulative Gross'] = df_2020_trim['Cumulative Gross'].str.replace('$', '')
df_2020_trim['Cumulative Gross'] = df_2020_trim['Cumulative Gross'].astype('int64', copy=True)

In [52]:
# Encode each holiday as a month
# holiday month list
month_name_list = []

# loop thru list
for i in range(len(df_2020_trim)):
    if df_2020_trim['Holiday'].iloc[i] in ["New Year's Eve", "Christmas Day"]:
        month_name_list.append('December')
    elif df_2020_trim['Holiday'].iloc[i] in ["Thanksgiving 5-Day Weekend"]:
        month_name_list.append('November')
    elif df_2020_trim['Holiday'].iloc[i] in ["Halloween", "Indigenous People's Day Weekend"]:
        month_name_list.append('October')
    elif df_2020_trim['Holiday'].iloc[i] in ["Labor Day Weekend"]:
        month_name_list.append('September')
    elif df_2020_trim['Holiday'].iloc[i] in ['Independence Day']:
        month_name_list.append('July')
    elif df_2020_trim['Holiday'].iloc[i] in ['Memorial Day Weekend']: 
        month_name_list.append('May')
    elif df_2020_trim['Holiday'].iloc[i] in ['Easter Weekend']: 
        month_name_list.append('April')
    elif df_2020_trim['Holiday'].iloc[i] in ["President's Day Weekend"]: 
        month_name_list.append('February')
    else:
        month_name_list.append('January')
        
display(len(month_name_list))

# Add lists to df as columns
df_2020_trim['Month'] = month_name_list
df_2020_trim['Year'] = [int(2020) for x in range(len(df_2020_trim))]

df_2020_trim = df_2020_trim[['Month', 'Year', 'Holiday', 'Cumulative Gross', 'Releases']]

df_2020_trim

12

,Month,Year,Holiday,Cumulative Gross,Releases
0,December,2020,New Year's Eve,2933369,17
1,December,2020,Christmas Day,10214882,27
2,November,2020,Thanksgiving 5-Day Weekend,20251599,36
3,October,2020,Halloween,3791211,35
4,October,2020,Indigenous People's Day Weekend,11259812,32
5,September,2020,Labor Day Weekend,19858242,25
6,July,2020,Independence Day,74261,9
7,May,2020,Memorial Day Weekend,240390,6
8,April,2020,Easter Weekend,3675,2
9,February,2020,President's Day Weekend,182920720,78


In [53]:
# Sum up the Gross Rev and # of Releases for each month
df_month_2020 = df_2020_trim[['Month', 'Cumulative Gross', 'Releases']].groupby(by='Month').sum()

# Add a Year Column
df_month_2020['Year'] = [int(2020) for x in range(len(df_month_2020))]

# Reset the Index and Trim the Dataframe
df_month_2020 = df_month_2020.reset_index()
df_month_2020 = df_month_2020[['Month', 'Year', 'Cumulative Gross', 'Releases']]

# Rename columns
df_month_2020 = df_month_2020.rename(columns = {'Cumulative Gross': 'Holiday Gross', 'Releases': 'Holiday Releases'})

df_month_2020

,Month,Year,Holiday Gross,Holiday Releases
0,April,2020,3675,2
1,December,2020,13148251,44
2,February,2020,182920720,78
3,January,2020,261809863,115
4,July,2020,74261,9
5,May,2020,240390,6
6,November,2020,20251599,36
7,October,2020,15051023,67
8,September,2020,19858242,25


### 2021 Holidays Table

In [54]:
holidays_2021_df

,Holiday,Cumulative Gross,Releases,Average
0,New Year's Eve,"$28,345,913",25,"$1,133,836"
1,Christmas Day,"$58,100,508",26,"$2,234,634"
2,Post-Thanksgiving Weekend,"$52,704,939",47,"$1,121,381"
3,Thanksgiving Weekend,"$96,526,140",51,"$1,892,669"
4,Thanksgiving 4-Day Weekend,"$116,694,008",38,"$3,070,894"
5,Thanksgiving,"$20,380,609",35,"$582,303"
6,Thanksgiving 5-Day Weekend,"$142,082,464",42,"$3,382,915"
7,Halloween,"$16,292,807",34,"$479,200"
8,Indigenous People's Day,"$14,257,686",29,"$491,644"
9,Indigenous People's Day Weekend,"$122,533,028",34,"$3,603,912"


In [55]:
# Drop redundant holiday rows
df_2021_trim = holidays_2021_df.drop([2,3,4,5,8,10,12,14,16,18,20])
# Reset Index
df_2021_trim.reset_index(inplace=True)
# Trim the columns
df_2021_trim = df_2021_trim[['Holiday', 'Cumulative Gross', 'Releases']]

df_2021_trim

,Holiday,Cumulative Gross,Releases
0,New Year's Eve,"$28,345,913",25
1,Christmas Day,"$58,100,508",26
2,Thanksgiving 5-Day Weekend,"$142,082,464",42
3,Halloween,"$16,292,807",34
4,Indigenous People's Day Weekend,"$122,533,028",34
5,Labor Day Weekend,"$139,020,923",48
6,Independence Day Weekend,"$87,171,208",46
7,Memorial Day Weekend,"$97,874,228",45
8,Easter Weekend,"$47,518,581",28
9,President's Day Weekend,"$13,132,939",32


In [56]:
# Convert Table
df_2021_trim['Cumulative Gross'] = df_2021_trim['Cumulative Gross'].str.replace(',', '')
df_2021_trim['Cumulative Gross'] = df_2021_trim['Cumulative Gross'].str.replace('$', '')
df_2021_trim['Cumulative Gross'] = df_2021_trim['Cumulative Gross'].astype('int64', copy=True)

In [57]:
# Encode each holiday as a month
# holiday month list
month_name_list = []

# loop thru list
for i in range(len(df_2020_trim)):
    if df_2021_trim['Holiday'].iloc[i] in ["New Year's Eve", "Christmas Day"]:
        month_name_list.append('December')
    elif df_2021_trim['Holiday'].iloc[i] in ["Thanksgiving 5-Day Weekend"]:
        month_name_list.append('November')
    elif df_2021_trim['Holiday'].iloc[i] in ["Halloween", "Indigenous People's Day Weekend"]:
        month_name_list.append('October')
    elif df_2021_trim['Holiday'].iloc[i] in ["Labor Day Weekend"]:
        month_name_list.append('September')
    elif df_2021_trim['Holiday'].iloc[i] in ['Independence Day', 'Independence Day Weekend']:
        month_name_list.append('July')
    elif df_2021_trim['Holiday'].iloc[i] in ['Memorial Day Weekend']: 
        month_name_list.append('May')
    elif df_2021_trim['Holiday'].iloc[i] in ['Easter Weekend']: 
        month_name_list.append('April')
    elif df_2021_trim['Holiday'].iloc[i] in ["President's Day Weekend"]: 
        month_name_list.append('February')
    else:
        month_name_list.append('January')
        
display(len(month_name_list))

# Add lists to df as columns
df_2021_trim['Month'] = month_name_list
df_2021_trim['Year'] = [int(2021) for x in range(len(df_2021_trim))]

df_2021_trim = df_2021_trim[['Month', 'Year', 'Holiday', 'Cumulative Gross', 'Releases']]

df_2021_trim

12

,Month,Year,Holiday,Cumulative Gross,Releases
0,December,2021,New Year's Eve,28345913,25
1,December,2021,Christmas Day,58100508,26
2,November,2021,Thanksgiving 5-Day Weekend,142082464,42
3,October,2021,Halloween,16292807,34
4,October,2021,Indigenous People's Day Weekend,122533028,34
5,September,2021,Labor Day Weekend,139020923,48
6,July,2021,Independence Day Weekend,87171208,46
7,May,2021,Memorial Day Weekend,97874228,45
8,April,2021,Easter Weekend,47518581,28
9,February,2021,President's Day Weekend,13132939,32


In [58]:
# Sum up the Gross Rev and # of Releases for each month
df_month_2021 = df_2021_trim[['Month', 'Cumulative Gross', 'Releases']].groupby(by='Month').sum()

# Add a Year Column
df_month_2021['Year'] = [int(2021) for x in range(len(df_month_2021))]

# Reset the Index and Trim the Dataframe
df_month_2021 = df_month_2021.reset_index()
df_month_2021 = df_month_2021[['Month', 'Year', 'Cumulative Gross', 'Releases']]

# Rename columns
df_month_2021 = df_month_2021.rename(columns = {'Cumulative Gross': 'Holiday Gross', 'Releases': 'Holiday Releases'})

df_month_2021

,Month,Year,Holiday Gross,Holiday Releases
0,April,2021,47518581,28
1,December,2021,86446421,51
2,February,2021,13132939,32
3,January,2021,17576635,43
4,July,2021,87171208,46
5,May,2021,97874228,45
6,November,2021,142082464,42
7,October,2021,138825835,68
8,September,2021,139020923,48


### 2022 Holidays Table

In [59]:
holidays_2022_df

,Holiday,Cumulative Gross,Releases,Average
0,New Year's Eve,"$28,295,543",31,"$912,759"
1,Christmas Day,"$43,796,821",28,"$1,564,172"
2,Christmas Weekend,"$142,814,390",33,"$4,327,708"
3,Post-Thanksgiving Weekend,"$52,669,526",54,"$975,361"
4,Thanksgiving Weekend,"$93,780,231",49,"$1,913,882"
5,Thanksgiving 4-Day Weekend,"$111,394,577",36,"$3,094,293"
6,Thanksgiving,"$17,723,816",31,"$571,736"
7,Thanksgiving 5-Day Weekend,"$134,030,714",37,"$3,622,451"
8,Halloween,"$7,440,404",34,"$218,835"
9,Indigenous People's Day,"$8,616,769",39,"$220,942"


In [60]:
# Drop redundant holiday rows
df_2022_trim = holidays_2022_df.drop([1,3,4,5,6,9,11,13,15,17,19,21,24])
# Reset Index
df_2022_trim.reset_index(inplace=True)
# Trim the columns
df_2022_trim = df_2022_trim[['Holiday', 'Cumulative Gross', 'Releases']]

df_2022_trim

,Holiday,Cumulative Gross,Releases
0,New Year's Eve,"$28,295,543",31
1,Christmas Weekend,"$142,814,390",33
2,Thanksgiving 5-Day Weekend,"$134,030,714",37
3,Halloween,"$7,440,404",34
4,Indigenous People's Day Weekend,"$67,265,387",45
5,Labor Day Weekend,"$71,172,772",55
6,Independence Day Weekend,"$223,580,059",40
7,Memorial Day Weekend,"$223,364,859",47
8,Easter Weekend,"$119,498,996",28
9,President's Day Weekend,"$111,455,709",51


In [61]:
# Convert Table
df_2022_trim['Cumulative Gross'] = df_2022_trim['Cumulative Gross'].str.replace(',', '')
df_2022_trim['Cumulative Gross'] = df_2022_trim['Cumulative Gross'].str.replace('$', '')
df_2022_trim['Cumulative Gross'] = df_2022_trim['Cumulative Gross'].astype('int64', copy=True)

In [62]:
# Encode each holiday as a month
# holiday month list
month_name_list = []

# loop thru list
for i in range(len(df_2020_trim)):
    if df_2022_trim['Holiday'].iloc[i] in ["New Year's Eve", "Christmas Day", "Christmas Weekend"]:
        month_name_list.append('December')
    elif df_2022_trim['Holiday'].iloc[i] in ["Thanksgiving 5-Day Weekend"]:
        month_name_list.append('November')
    elif df_2022_trim['Holiday'].iloc[i] in ["Halloween", "Indigenous People's Day Weekend"]:
        month_name_list.append('October')
    elif df_2022_trim['Holiday'].iloc[i] in ["Labor Day Weekend"]:
        month_name_list.append('September')
    elif df_2022_trim['Holiday'].iloc[i] in ['Independence Day', 'Independence Day Weekend']:
        month_name_list.append('July')
    elif df_2022_trim['Holiday'].iloc[i] in ['Memorial Day Weekend']: 
        month_name_list.append('May')
    elif df_2022_trim['Holiday'].iloc[i] in ['Easter Weekend']: 
        month_name_list.append('April')
    elif df_2022_trim['Holiday'].iloc[i] in ["President's Day Weekend"]: 
        month_name_list.append('February')
    else:
        month_name_list.append('January')
        
display(len(month_name_list))

# Add lists to df as columns
df_2022_trim['Month'] = month_name_list
df_2022_trim['Year'] = [int(2022) for x in range(len(df_2022_trim))]

df_2022_trim = df_2022_trim[['Month', 'Year', 'Holiday', 'Cumulative Gross', 'Releases']]

df_2022_trim

12

,Month,Year,Holiday,Cumulative Gross,Releases
0,December,2022,New Year's Eve,28295543,31
1,December,2022,Christmas Weekend,142814390,33
2,November,2022,Thanksgiving 5-Day Weekend,134030714,37
3,October,2022,Halloween,7440404,34
4,October,2022,Indigenous People's Day Weekend,67265387,45
5,September,2022,Labor Day Weekend,71172772,55
6,July,2022,Independence Day Weekend,223580059,40
7,May,2022,Memorial Day Weekend,223364859,47
8,April,2022,Easter Weekend,119498996,28
9,February,2022,President's Day Weekend,111455709,51


In [63]:
# Sum up the Gross Rev and # of Releases for each month
df_month_2022 = df_2022_trim[['Month', 'Cumulative Gross', 'Releases']].groupby(by='Month').sum()

# Add a Year Column
df_month_2022['Year'] = [int(2021) for x in range(len(df_month_2022))]

# Reset the Index and Trim the Dataframe
df_month_2022 = df_month_2022.reset_index()
df_month_2022 = df_month_2022[['Month', 'Year', 'Cumulative Gross', 'Releases']]

# Rename columns
df_month_2022 = df_month_2022.rename(columns = {'Cumulative Gross': 'Holiday Gross', 'Releases': 'Holiday Releases'})

df_month_2022

,Month,Year,Holiday Gross,Holiday Releases
0,April,2021,119498996,28
1,December,2021,171109933,64
2,February,2021,111455709,51
3,January,2021,125403205,63
4,July,2021,223580059,40
5,May,2021,223364859,47
6,November,2021,134030714,37
7,October,2021,74705791,79
8,September,2021,71172772,55


### Inflation Adjustment to Top 200 table

In [64]:
from pathlib import Path

In [65]:
# list of inflation multipliers 1983-2022, cpi-U yearly change
inflation_df = pd.read_csv('../inflation_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../inflation_data.csv'

In [ ]:
len(inflation_df)

In [ ]:
%pprint

In [ ]:
# make a list of individual year inflation multipliers
inflation_list = inflation_df['inflation_multiplier'].values.tolist()

display(inflation_list)

# reverse the order of the list
inflation_list.reverse()
print('-'*25)
display(inflation_list)

In [ ]:
[2022 for x in range(200)] 